In [ ]:
import concurrent.futures
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import IPython.display as ipd
from tqdm import tqdm
import sounddevice as sd
import soundfile as sf
import librosa
import h5py
import json
import librosa
from librosa.core import resample
import pyrubberband as pyrb


In [ ]:
import rubberband

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
#musdb in hand
import musdb
#mus = musdb.DB('/home/pedro.lopes/data/dataset/musdb18')
mus = musdb.DB('/home/pedro.lopes/data/dataset/musdb18hq',is_wav=True)
mus

In [ ]:
f,t,stft_data = signal.stft(mus[0].audio[:,0] + mus[0].audio[:,1])
log_stft = np.log(np.abs(stft_data)+1e-7)
plt.pcolormesh(log_stft)

In [ ]:
!export MEDLEYDB_PATH="nfs/home/pedro.lopes/data/dataset/MedleyDB_V2/V2/"

In [ ]:
from glob import glob
%cd /home/pedro.lopes/data/dataset/MedleyDB_V2/V2/
names = glob("*")
%cd /home/pedro.lopes/pf/code
import medleydb as mdb
vocal_paths =[]
medley_path = '/home/pedro.lopes/data/dataset/MedleyDB_V2/V2/'
instruments = ['male speaker','female speaker','female singer', 'male singer','vocalists','male rapper','female rapper','male screamer','female screamer']
mtrack_generator = mdb.load_all_multitracks()
medley_vocal_audios = []
medley_instrumental_audios =[]
medley_complete_audios=[]
for name in names:
    track = mdb.MultiTrack(name)
    track_vocal_paths =[]
    track_instrumental_paths = []
    path = medley_path + name +'/' + name +'_MIX'+ '.wav'
    directory = medley_path + name +'/' + name + '_STEMS/'
    %cd $directory
    stem_paths = glob("*.wav")
    mix_audio,sr = librosa.load(path,sr=None)
    vocal_audio = np.zeros_like(mix_audio)
    instrumental_audio = np.zeros_like(mix_audio)
    for key, stem in track.stems.items():
        is_vocal=True
        for stem_instrument in stem.instrument:
            if stem_instrument not in instruments:
                is_vocal=False
        if is_vocal:
            print(stem.instrument)
            vocal_path = name + '_STEM_' + '{:02}'.format(stem.stem_idx) + '.wav'
            y,sr = librosa.load(vocal_path,sr=None)
            if sr!=44100:
                print(sr)
                y = resample(y,sr,44100)
            vocal_audio+=y
            track_vocal_paths.append(vocal_path)
    track_instrumental_paths = [x for x in stem_paths if x not in track_vocal_paths]
    for instrumental_path in track_instrumental_paths:
        y,sr = librosa.load(instrumental_path,sr=None)
        if sr!=44100:
            y = resample(y,sr,44100)
        instrumental_audio+=y
    if vocal_audio.any():
        medley_vocal_audios.append(vocal_audio)
        medley_instrumental_audios.append(instrumental_audio)
        medley_complete_audios.append(vocal_audio + instrumental_audio)



print(str(len(medley_vocal_audios)) + " Musicas com vocal carregadas")

In [ ]:
!export MEDLEYDB_PATH="nfs/home/pedro.lopes/data/dataset/MedleyDB_V3/"
%cd /home/pedro.lopes/data/dataset/MedleyDB_V3/
names = glob("*")
%cd /home/pedro.lopes/pf/code
import medleydb as mdb
vocal_paths =[]
medley_path = '/home/pedro.lopes/data/dataset/MedleyDB_V3/'
instruments = ['male speaker','female speaker','female singer', 'male singer','vocalists','male rapper','female rapper','male screamer','female screamer']
mtrack_generator = mdb.load_all_multitracks()
medley3_vocal_audios = []
medley3_instrumental_audios =[]
medley3_complete_audios=[]
for name in names:
    try:
        track = mdb.MultiTrack(name)
    except:
        print('nao achei metadado: '+ name)
        continue
    track_vocal_paths =[]
    track_instrumental_paths = []
    path = medley_path + name +'/' + name + '_STEMS/' + name + '_STEM_01.wav'
    directory = medley_path + name +'/' + name + '_STEMS/'
    %cd $directory
    stem_paths = glob("*.wav")
    mix_audio,sr = librosa.load(path,sr=None)
    if sr!=44100:
        print(sr)
        mix_audio = resample(mix_audio,sr,44100)
    vocal_audio = np.zeros_like(mix_audio)
    instrumental_audio = np.zeros_like(mix_audio)
    for key, stem in track.stems.items():
        is_vocal=True
        for stem_instrument in stem.instrument:
            if stem_instrument not in instruments:
                is_vocal=False
        if is_vocal:
            print(stem.instrument)
            vocal_path = name + '_STEM_' + '{:02}'.format(stem.stem_idx) + '.wav'
            y,sr = librosa.load(vocal_path,sr=None)
            if sr!=44100:
                print(sr)
                y = resample(y,sr,44100)
            try:
                vocal_audio+=y
            except:
                print('Não consegui vocal: ' + vocal_path)
                pass
            track_vocal_paths.append(vocal_path)
    track_instrumental_paths = [x for x in stem_paths if x not in track_vocal_paths]
    for instrumental_path in track_instrumental_paths:
        y,sr = librosa.load(instrumental_path,sr=None)
        if sr!=44100:
            y = resample(y,sr,44100)
        try:
            instrumental_audio+=y
            
                
                
        except:
            print('Não consegui instrumental: ' + instrumental_path)
            pass
    if vocal_audio.any():
        medley3_vocal_audios.append(vocal_audio)
        medley3_instrumental_audios.append(instrumental_audio)
        medley3_complete_audios.append(vocal_audio + instrumental_audio)



print(str(len(medley3_vocal_audios)) + " Musicas com vocal carregadas")

In [ ]:
%cd /home/pedro.lopes/data/dataset/MedleyDB_V3/
track2 = mdb.MultiTrack(name)
track2._meta_path

In [ ]:
print(name)

In [ ]:
y,sr = librosa.load('../audio_data/train/sample_1310_-1_acc.wav',sr=None)
ipd.Audio(y,rate=sr)

In [ ]:

out = pyrb.pitch_shift(y=y,sr=sr,n_steps=-8,rbargs={'--formant': '-F'})
ipd.Audio(out,rate=sr)

In [ ]:
ccmixter_path = '/home/pedro.lopes/data/dataset/ccmixter_corpus/'
%cd $ccmixter_path
names = glob('*')
names.remove('README')
ccmixter_vocal_audios = []
ccmixter_instrumental_audios = []
ccmixter_complete_audios = []
for name in names:
    directory = ccmixter_path + name + '/'
    instrumental_path = directory + 'source-01.wav'
    vocal_path = directory + 'source-02.wav'
    mix_path = directory + 'mix.wav'
    y,sr = librosa.load(vocal_path,sr=None)
    ccmixter_vocal_audios.append(y)
    y,sr = librosa.load(instrumental_path,sr=None)
    ccmixter_instrumental_audios.append(y)
    y,sr = librosa.load(mix_path,sr=None)
    ccmixter_complete_audios.append(y)
print(str(len(ccmixter_vocal_audios)) + " Musicas com vocal carregadas")

In [ ]:
mir1k_path = '/home/pedro.lopes/data/dataset/MIR-1K/Wavfile/'
%cd $mir1k_path
vocal_path = mir1k_path +'amy_8_01.wav'
names = glob('*')
mir1k_vocal_audios = []
mir1k_instrumental_audios = []
mir1k_complete_audios = []
for name in names:    
    y,sr = librosa.load(vocal_path,sr=None,mono=False)
    instrumental_audio=y[0]
    vocal_audio = y[1]
    mir1k_vocal_audios.append(vocal_audio)
    mir1k_instrumental_audios.append(instrumental_audio)
    mir1k_complete_audios.append(vocal_audio + instrumental_audio)
print(sr)
print(str(len(mir1k_vocal_audios)) + " Musicas com vocal carregadas")

In [ ]:
len(mus)

In [ ]:
ipd.Audio(y[1],rate=sr)

In [ ]:
y = resample(np.asfortranarray(mir1k_vocal_audios[0]),16000,44100)
ipd.Audio(y,rate = 44100)

In [ ]:
ipd.Audio(medley3_complete_audios[0],rate=sr)

In [ ]:

acc = mus[0].targets['accompaniment'].audio[:,0] + mus[0].targets['accompaniment'].audio[:,1]
vocals = mus[0].targets['vocals'].audio[:,0] + mus[0].targets['vocals'].audio[:,1]
al = mus[0].audio[:,0] + mus[0].audio[:,1]
#audio1 = mus[0].audio
ipd.Audio(al - acc ,rate=mus[0].rate)

In [ ]:
ipd.Audio(al,rate=mus[0].rate)

In [ ]:
vocals = np.asfortranarray(mir1k_vocal_audios[0])
resampled = resample(vocals,16000,8192)
ipd.Audio(resampled,rate=8192)

In [ ]:
'''track_list = []
vocal_list = []
stft_list = []
vocal_stft_list = []
num_seg = 25
num_samples = num_seg*mus[0].rate
num_freq = 512
shape = (len(mus),num_freq+1,int(np.ceil(num_samples/(2*num_freq))*2+1))
stft_list = np.zeros(shape,dtype=complex)
vocal_stft_list = np.zeros(shape,dtype=complex)'''

'''for i in tqdm(range(len(mus))):
    track = mus[i]
    interval_start = int(np.random.uniform(0, len(track.audio)-num_samples))
    f,t, stft_data = signal.stft(track.audio[interval_start:interval_start + num_samples,0] + track.audio[interval_start:interval_start + num_samples,1], track.rate, nperseg=(2*num_freq),noverlap=num_freq)
    #f,t, stft_data = signal.stft(track.audio[:,0] + track.audio[:,1], track.rate, nperseg=(2*num_freq),noverlap=num_freq)
    stft_list[i] = stft_data
    f,t, vocal_stft =  signal.stft(track.targets['vocals'].audio[interval_start:interval_start + num_samples,0] + track.targets['vocals'].audio[interval_start:interval_start + num_samples,1], track.rate,nperseg=1024,noverlap=512)
    #f,t, vocal_stft =  signal.stft(track.targets['vocals'].audio[:,0] + track.targets['vocals'].audio[:,1], track.rate,nperseg=1024,noverlap=512)
    f,t, acc_stft = signal.stft(track.targets['accompaniment'].audio[interval_start:interval_start + num_samples,0] + track.targets['accompaniment'].audio[interval_start:interval_start + num_samples,0], track.rate,nperseg=1024,noverlap=512)
    vocal_stft_list[i] = vocal_stft
'''
    

In [ ]:
def append_to_dataset(datapath,dataset_name,new_data):
  with h5py.File(datapath, mode='a') as h5f:
    dset = h5f[dataset_name]
    shape_old = dset.shape
    shape_new = list(dset.shape)
    shape_new[0] += new_data.shape[0]
    shape_new = tuple(shape_new)
    dset.resize(shape_new)
    #print(shape_old,shape_new)
    dset[shape_old[0]:shape_new[0],:] = new_data
    h5f.flush()

In [ ]:
hf_train.close()
hf_test.close()

In [ ]:
track_list = []
vocal_list = []
stft_list = []
vocal_stft_list = []

num_freq = 512
%cd ~/pf/code
# datapath = '../data/4datasets_16khz_train.h5'
# datapath_test = '../data/4datasets_16khz_test.h5'
# hf_train = h5py.File(datapath, 'w')
# hf_test = h5py.File(datapath_test, 'w')

#hf.create_dataset('Y_train_acc',chunks=True,maxshape=(None,513))
path = '../norm_data_full.json'
norm_data = {}


len_others = len(mir1k_complete_audios) + len(ccmixter_complete_audios) + len(medley_complete_audios) + len(medley3_complete_audios)
start = 0#len(mus) + len(mir1k_complete_audios) + len(ccmixter_complete_audios) + len(medley_complete_audios)
with concurrent.futures.ProcessPoolExecutor() as executor:
    for i in tqdm(range(start,len(mus) + len_others)):
    # for i in tqdm(range(start,len(mus))):
        rate_down = 44100
        mode = 'train'
        rate = 44100
        if i<100:
            acc = mus[i].targets['accompaniment'].audio[:,0] + mus[i].targets['accompaniment'].audio[:,1]
            vocals = mus[i].targets['vocals'].audio[:,0] + mus[i].targets['vocals'].audio[:,1]
            al = mus[i].audio[:,0] + mus[i].audio[:,1]
        if i>=100 and i<150:
            acc = mus[i].targets['accompaniment'].audio[:,0] + mus[i].targets['accompaniment'].audio[:,1]
            vocals = mus[i].targets['vocals'].audio[:,0] + mus[i].targets['vocals'].audio[:,1]
            al = mus[i].audio[:,0] + mus[i].audio[:,1]
            mode='test'
        if i>=len(mus) and i<(len(mus)+len(mir1k_complete_audios)):
            rate = 16000
            aux_index = i -len(mus)
            acc = mir1k_instrumental_audios[aux_index]
            vocals = mir1k_vocal_audios[aux_index]
            al = mir1k_complete_audios[aux_index]
        if i>=(len(mus)+len(mir1k_complete_audios)) and i<(len(mus)+len(mir1k_complete_audios) + len(ccmixter_complete_audios)):
            rate = 44100
            aux_index = i - (len(mus)+len(mir1k_complete_audios))
            acc = ccmixter_instrumental_audios[aux_index]
            vocals = ccmixter_vocal_audios[aux_index]
            al = ccmixter_complete_audios[aux_index]
        if i>=(len(mus)+len(mir1k_complete_audios) + len(ccmixter_complete_audios)) and i<(len(mus)+len(mir1k_complete_audios) + len(ccmixter_complete_audios) + len(medley_complete_audios)):
            aux_index = i - (len(mus)+len(mir1k_complete_audios) + len(ccmixter_complete_audios))
            acc = medley_instrumental_audios[aux_index]
            vocals = medley_vocal_audios[aux_index]
            al = medley_complete_audios[aux_index]
        if i>=(len(mus)+len(mir1k_complete_audios) + len(ccmixter_complete_audios) + len(medley_complete_audios)):
            aux_index = i - (len(mus)+len(mir1k_complete_audios) + len(ccmixter_complete_audios) + len(medley_complete_audios))
            acc = medley3_instrumental_audios[aux_index]
            vocals = medley3_vocal_audios[aux_index]
            al = medley3_complete_audios[aux_index]
        acc = np.asfortranarray(acc)
        vocals = np.asfortranarray(vocals)
        al = np.asfortranarray(al)
        if rate != rate_down:
            acc_resampled = resample(acc,rate,rate_down)
            vocals_resampled = resample(vocals,rate,rate_down)
            al_resampled = resample(al,rate,rate_down)
        else:
            acc_resampled = acc
            vocals_resampled = vocals
            al_resampled = al
        if mode=='train':
            audio_path = '/home/pedro.lopes/data/audio_data/train/'
        else:
            audio_path = '/home/pedro.lopes/data/audio_data/test/'


        audio_acc = acc_resampled
        audio_vocals = vocals_resampled
        audio_al = al_resampled
        librosa.output.write_wav(audio_path + 'sample_' + str(i) + '_vocals.wav',y=audio_vocals,sr=rate_down)
        librosa.output.write_wav(audio_path + 'sample_' + str(i) + '_acc.wav',y=audio_acc,sr=rate_down)
        librosa.output.write_wav(audio_path + 'sample_' + str(i) + '_mix.wav',y=audio_al,sr=rate_down)

#         for j in range(-3,4,1):
#             if j==0:
#                 audio_acc = acc_resampled
#                 audio_vocals = vocals_resampled
#                 audio_al = vocals_resampled + acc_resampled
#             else:
#                 audio_acc = pyrb.pitch_shift(y=acc_resampled,sr=rate_down,n_steps=j,rbargs={'--formant': '-F'})
#                 audio_vocals = pyrb.pitch_shift(y=vocals_resampled,sr=rate_down,n_steps=j,rbargs={'--formant': '-F'})
#                 audio_al = audio_vocals + audio_acc
#             librosa.output.write_wav(audio_path + 'sample_' + str(i) + '_' + str(j) + '_vocals.wav',y=audio_vocals,sr=rate_down)
#             librosa.output.write_wav(audio_path + 'sample_' + str(i) + '_' + str(j) + '_acc.wav',y=audio_acc,sr=rate_down)
#             librosa.output.write_wav(audio_path + 'sample_' + str(i) + '_' + str(j) + '_mix.wav',y=audio_al,sr=rate_down)
#         time_stretches =  [0.5, 0.93, 1, 1.07, 1.15]
#         for ts in time_stretches:
#             if ts==1:
#                 audio_acc = acc_resampled
#                 audio_vocals = vocals_resampled
#                 audio_al = vocals_resampled + acc_resampled
#             else:


#                 audio_acc = pyrb.time_stretch(y=acc_resampled,sr=rate_down,rate=ts)
#                 audio_vocals = pyrb.time_stretch(y=vocals_resampled,sr=rate_down,rate=ts)
#                 audio_al = audio_vocals + audio_acc
#             librosa.output.write_wav(audio_path + 'sample_' + str(i) + '_' + str(ts) + 't_vocals.wav',y=audio_vocals,sr=rate_down)
#             librosa.output.write_wav(audio_path + 'sample_' + str(i) + '_' + str(ts) + 't_acc.wav',y=audio_acc,sr=rate_down)
#             librosa.output.write_wav(audio_path + 'sample_' + str(i) + '_' + str(ts) + 't_mix.wav',y=audio_al,sr=rate_down)


    #     acc_resampled_shifted = get_audio_with_shifts(acc_resampled,rate_down)
    #     vocals_resampled_shifted = get_audio_with_shifts(vocals_resampled,rate_down)
    #     al_resampled_shifted = get_audio_with_shifts(al_resampled,rate_down)

    #     _,_,stft_data = signal.stft(al_resampled, rate_down, nperseg=(2*num_freq),noverlap=num_freq) # lembrar que a função cospe f,t, data
    #     #f,t, stft_data = signal.stft(track.audio[:,0] + track.audio[:,1], track.rate, nperseg=(2*num_freq),noverlap=num_freq)
    #     _,_,acc_stft = signal.stft(acc_resampled, rate_down,nperseg=1024,noverlap=512)
    #     _,_,vocal_stft =  signal.stft(vocals_resampled, rate_down,nperseg=1024,noverlap=512)
    #     if i==0:
    #         hf_train.create_dataset('X_train',chunks=True,maxshape=(None,513),data=stft_data.T)
    #         hf_train.create_dataset('Y_train_vocal',chunks=True,maxshape=(None,513),data=vocal_stft.T)
    #         hf_train.create_dataset('Y_train_acc',chunks=True,maxshape=(None,513),data=acc_stft.T)

    #         norm_data['X_max'] = 0
    #         norm_data['X_min'] = 1000
    #         with open(path, 'w') as fp:
    #           json.dump(norm_data, fp)
    #         hf_train.close()
    #     if i==100:
    #         hf_test.create_dataset('X_test',chunks=True,maxshape=(None,513),data=stft_data.T)
    #         hf_test.create_dataset('Y_test_vocal',chunks=True,maxshape=(None,513),data=vocal_stft.T)
    #         hf_test.create_dataset('Y_test_acc',chunks=True,maxshape=(None,513),data=acc_stft.T)
    #         hf_test.close()


    #     if mode == 'train':
    #         append_to_dataset(datapath=datapath,dataset_name='X_train',new_data=stft_data.T)
    #         append_to_dataset(datapath=datapath,dataset_name='Y_train_vocal',new_data=vocal_stft.T)
    #         append_to_dataset(datapath=datapath,dataset_name='Y_train_acc',new_data=acc_stft.T)
    #         if(np.max(np.abs(stft_data[2]))>=norm_data['X_max']):
    #             norm_data['X_max'] = np.log(np.max(np.abs(stft_data))+1e-7)
    #         with open(path, 'w') as fp:
    #             json.dump(norm_data, fp)
    #         if(np.min(np.abs(stft_data[2]))<=norm_data['X_min']):
    #             norm_data['X_min'] = np.log(np.min(np.abs(stft_data))+1e-7)
    #         with open(path, 'w') as fp:
    #             json.dump(norm_data, fp)
    #     else:
    #         append_to_dataset(datapath=datapath_test,dataset_name='X_test',new_data=stft_data.T)
    #         append_to_dataset(datapath=datapath_test,dataset_name='Y_test_vocal',new_data=vocal_stft.T)
    #         append_to_dataset(datapath=datapath_test,dataset_name='Y_test_acc',new_data=acc_stft.T)

    #     #vocal_stft_list[i] = vocal_stft

In [ ]:
i

In [ ]:
y,sr = librosa.load(audio_path + 'sample_' + str(1) + '_vocals.wav',sr=None,mono=False)
f,t,stft_data = signal.stft(y)
plt.pcolormesh(np.log(np.abs(stft_data)+1e-7))

In [ ]:
ipd.Audio(y,sr)

In [ ]:
h5 = h5py.File(datapath, 'r')
json_path = '../len_full_samples.json'
X_train_true = h5.get('X_train')
train_len = {}
train_len['len'] = X_train_true.shape
with open(json_path, 'w') as fp:
    json.dump(train_len, fp)


In [ ]:
import json
norm_data = {}
norm_data['X_max'] = max_X_train
norm_data['X_min'] = min_X_train
path = '/content/drive/My Drive/Projeto Final/data/norm_data_full.json' 
with open(path, 'w') as fp:
    json.dump(norm_data, fp)

In [ ]:
hf = h5py.File(datapath, 'r')
plt.pcolormesh(np.abs(np.log(hf['X_train'][0:128].T+1e-8)))

In [ ]:
stft_list = np.hstack(stft_list).T
vocal_stft_list = np.hstack(vocal_stft_list).T
acc_stft_list = np.hstack(acc_stft_list).T

In [ ]:
teste.shape

In [ ]:
ipd.Audio(track.audio[0:num_samples,0],rate=mus[0].rate)

In [ ]:
import matplotlib.cm as cm
num_seg = 10
num_samples = num_seg*mus[0].rate
f,t, stft_data = signal.stft(mus[0].targets['vocals'].audio[0:num_samples,0] + mus[0].targets['vocals'].audio[0:num_samples,1], mus[0].rate,nperseg=1024,noverlap=512)
print(len(stft_data[1]))

i = 3

freq_bins = 513
plt.pcolormesh(t, f, np.log(np.abs(stft_data)+1e-30),cmap=cm.gray)
xmin =0
xmax =10
ymin =0
ymax = 20000

plt.axis([xmin, xmax, ymin, ymax])

In [ ]:

hf = h5py.File('/content/drive/My Drive/Projeto Final/data/data_context_2stems.h5', 'w')


hf.create_dataset('X_train', data=stft_list)
hf.create_dataset('Y_train_vocal', data=vocal_stft_list)
hf.create_dataset('Y_train_acc', data=acc_stft_list)

hf.close()
path = '/content/drive/My Drive/Projeto Final/data/len_raw.npy' 
np.save(path,len(stft_list))

print('terminei!')

In [ ]:
def normalize(x,save=False):
    scaled_x = (x - np.mean(x))/(np.abs(np.std(x))+1e-8)

    if save:
      return scaled_x, np.mean(x), np.std(x)
    #scaled_x = scaled_x - np.mean(scaled_x)
    return scaled_x

In [ ]:
def normalize_01(x,save=False):
    scaled_x = (x - np.min(x))/(np.max(x) - np.min(x)+1e-8)

    if save:
      return scaled_x, np.min(x), np.max(x)
    #scaled_x = scaled_x - np.mean(scaled_x)
    return scaled_x

In [ ]:
import sys
h5 = h5py.File('/content/drive/My Drive/Projeto Final/data/data_context.h5', 'r')
X_train_true = h5.get('X_train')
X_train_true.shape

In [ ]:
X_train_true[0:500].shape

In [ ]:
append_to_dataset('test.h5','X_train',X_train_true[500:1000])

In [ ]:
import json

norm_data ={}

h5 = h5py.File('/content/drive/My Drive/Projeto Final/data/full_context.h5', 'r')

# X_normal,norm_data['X_mean'],norm_data['X_std'] = normalize(np.log(np.abs(h5.get('X_train'))+1e-7),save =True)
# y_normal,norm_data['y_mean'],norm_data['y_std'] = normalize(np.log(np.abs(h5.get('Y_train'))+1e-7),save = True)
# hf = h5py.File('/content/drive/My Drive/Projeto Final/data/data_normalized_mean.h5', 'w')


print(np.min(h5.get('X_train')))

# hf.create_dataset('X_train', data=X_normal)
# hf.create_dataset('Y_train', data=y_normal)

hf.close()
h5.close()




In [ ]:
path = '/content/drive/My Drive/Projeto Final/data/norm_data_mean.json' 
with open(path, 'w') as fp:
    json.dump(norm_data, fp)

In [ ]:
import json

norm_data ={}

h5 = h5py.File('/content/drive/My Drive/Projeto Final/data/full_context.h5', 'r')

X_normal,norm_data['X_min'],norm_data['X_max'] = normalize_01(np.log(np.abs(h5.get('X_train'))+1e-7),save =True)
y_vocal_normal,norm_data['y_max_voc'],norm_data['y_min_voc'] = normalize_01(np.log(np.abs(h5.get('Y_train_vocal'))+1e-7),save = True)
#y_acc_normal,norm_data['y_max_acc'],norm_data['y_min_acc'] = normalize_01(np.log(np.abs(h5.get('Y_train_acc'))+1e-7),save = True)
hf = h5py.File('/content/drive/My Drive/Projeto Final/data/data_normalized_01_full.h5', 'w')


hf.create_dataset('X_train', data=X_normal)
hf.create_dataset('Y_train_vocal', data=y_vocal_normal)
#hf.create_dataset('Y_train_acc', data=y_acc_normal)

hf.close()
h5.close()




In [ ]:
path = '/content/drive/My Drive/Projeto Final/data/norm_data_01.json' 
with open(path, 'w') as fp:
    json.dump(norm_data, fp)

In [ ]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=32, dim=(32,32,32), n_channels=1,
                shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()
        self.path = ''

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = np.array(np.log(np.abs(X_train[ID])+1e-10)).reshape((*self.dim, self.n_channels))
            X[i,] = X[i,]
            # Store class
            y[i] = np.log(np.abs(Y_train[ID])+1e-10)
        
        X = normalize(X)
        y = normalize(y)


        return X, y

In [ ]:
train_data_groups.shape

In [ ]:
target_groups[0].shape

In [ ]:
a = train_data_groups

plt.pcolormesh(t[i:i+sample_len], f, np.abs(a[0]))

In [ ]:
plt.pcolormesh(np.abs(teste[:,0:863]))

In [ ]:
a.shape

In [ ]:
num_samples/5

In [ ]:
with tf.device('/device:GPU:0'):
  model = Sequential()
  model.add(Conv2D(16, (3,3), padding='same', input_shape=(freq_bins, sample_len, 1)))
  model.add(LeakyReLU())
  model.add(Conv2D(16, (3,3), padding='same'))
  model.add(LeakyReLU())
  model.add(MaxPooling2D(pool_size=(3,3)))
  model.add(Dropout(0.25))
  model.add(Conv2D(16, (3,3), padding='same'))
  model.add(LeakyReLU())
  model.add(Conv2D(16, (3,3), padding='same'))
  model.add(LeakyReLU())
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(64))
  model.add(LeakyReLU())
  model.add(Dropout(0.5))
  model.add(Dense(freq_bins, activation='relu'))
  model.compile(loss=keras.losses.mse, optimizer='adam', metrics=['mse'])

In [ ]:
from keras_tqdm import TQDMNotebookCallback

with tf.device('/device:GPU:0'):
  history = model.fit(np.abs(train_data_groups).reshape(target_groups.shape[0],freq_bins,sample_len,1),np.abs(target_groups),batch_size = 32,epochs = 30)

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

In [ ]:
a = model.predict(np.abs(train_data_groups[0:1*len(stft_list[0][0])].reshape(len(stft_list[0][0]),513,11,1)))
test = np.transpose(a)
test_list = []
for i in range(len(test)):
  test_list.append(np.roll(test[i],2))
test = np.array(test_list)

In [ ]:
len(stft_list[0][0])

In [ ]:
result_stft = np.multiply(test,stft_list[0])
plt.pcolormesh(t, f, np.abs(result_stft))
xmin =0
xmax = 3
ymin =0
ymax = 3000

plt.axis([xmin, xmax, ymin, ymax])


In [ ]:
plt.pcolormesh(t, f, np.abs(vocal_stft_list[2]))
xmin =0
xmax = 30
ymin =0
ymax = 3000

plt.axis([xmin, xmax, ymin, ymax])

In [ ]:
audio1 = signal.istft(vocal_stft_list[863:863*2].T,fs =mus[0].rate)
audio1

In [ ]:
ipd.Audio(audio1[1],rate=mus[0].rate)

In [ ]:
ipd.Audio(mus[10].audio[:,0] + mus[10].audio[:,1],rate=mus[0].rate)